<a href="https://colab.research.google.com/github/Blind5518/pyspark_books_authors/blob/main/books_authors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1. Загрузка библиотек,  импорт и чтение данных**

In [5]:
# Установка pyspark и создание SparkSession
!pip install pyspark

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import DateType, DoubleType

spark = SparkSession.builder \
    .appName("BooksAuthorsAnalysis") \
    .getOrCreate()

In [6]:
# Чтение CSV в DataFrame (предполагается, что файлы находятся в рабочей директории)
books_df = spark.read.csv("/books.csv", header=True, inferSchema=False)
authors_df = spark.read.csv("/authors.csv", header=True, inferSchema=False)

# Быстрый просмотр первых строк
display(books_df.limit(5).toPandas())
display(authors_df.limit(5).toPandas())

,book_id,title,author_id,genre,price,publish_date
0,1,Book_1,2,Mystery,73.57,1980-12-31
1,2,Book_2,1,Non-Fiction,41.1,1982-12-31
2,3,Book_3,10,Fiction,10.63,1984-12-31
3,4,Book_4,9,Non-Fiction,46.31,1986-12-31
4,5,Book_5,7,Science,31.13,1988-12-31


,author_id,name,birth_date,country
0,1,Author_1,1960-12-31,India
1,2,Author_2,1965-12-31,Canada
2,3,Author_3,1970-12-31,USA
3,4,Author_4,1975-12-31,UK
4,5,Author_5,1980-12-31,USA


**2. Обработка данных**

In [7]:
# Преобразование publish_date и birth_date в формат даты
books_df = books_df.withColumn(
    "publish_date",
    F.to_date("publish_date", "yyyy-MM-dd")
)

authors_df = authors_df.withColumn(
    "birth_date",
    F.to_date("birth_date", "yyyy-MM-dd")
)

# Перевод price к числовому типу
books_df = books_df.withColumn("price", F.col("price").cast(DoubleType()))

In [8]:
# Объединение данных по author_id
books_authors_df = books_df.join(authors_df, on="author_id", how="inner")

books_authors_df.limit(5).show(truncate=False)

+---------+-------+------+-----------+-----+------------+---------+----------+---------+
|author_id|book_id|title |genre      |price|publish_date|name     |birth_date|country  |
+---------+-------+------+-----------+-----+------------+---------+----------+---------+
|2        |1      |Book_1|Mystery    |73.57|1980-12-31  |Author_2 |1965-12-31|Canada   |
|1        |2      |Book_2|Non-Fiction|41.1 |1982-12-31  |Author_1 |1960-12-31|India    |
|10       |3      |Book_3|Fiction    |10.63|1984-12-31  |Author_10|2005-12-31|India    |
|9        |4      |Book_4|Non-Fiction|46.31|1986-12-31  |Author_9 |2000-12-31|Australia|
|7        |5      |Book_5|Science    |31.13|1988-12-31  |Author_7 |1990-12-31|USA      |
+---------+-------+------+-----------+-----+------------+---------+----------+---------+



**3. Анализ данных**

In [9]:
# Топ-5 авторов по общей выручке
top5_revenue = (
    books_authors_df
        .groupBy("author_id", "name")
        .agg(F.sum("price").alias("total_revenue"))
        .orderBy(F.desc("total_revenue"))
        .limit(5)
)


# Количество книг в каждом жанре
books_per_genre = (
    books_authors_df
        .groupBy("genre")
        .agg(F.count("book_id").alias("books_count"))
        .orderBy(F.desc("books_count"))
)


# Средняя цена книг по авторам
avg_price = (
    books_authors_df
        .groupBy("author_id", "name")
        .agg(F.round(F.avg("price"), 2).alias("avg_price"))
        .orderBy(F.desc("avg_price"))
)


# Книги после 2000 года, сортировка по цене
books_after_2000 = (
    books_authors_df
        .filter(F.col("publish_date") > F.to_date(F.lit("2000-01-01")))
        .select("book_id", "title", "name", "genre", "price", "publish_date")
        .orderBy(F.desc("price"))
)

**4. Вывод результатов**

In [10]:
print("Топ-5 авторов, книги которых принесли наибольшую выручку:")
top5_revenue.show(truncate=False)

print("Количество книг в каждом жанре:")
books_per_genre.show(truncate=False)

print("Среднюя цена книги по каждому автору:")
avg_price.show(truncate=False)

print("Книги, опубликованные после 2000 года, отсортиртированные по цене:")
books_after_2000.show(truncate=False)

Топ-5 авторов, книги которых принесли наибольшую выручку:
+---------+--------+-------------+
|author_id|name    |total_revenue|
+---------+--------+-------------+
|2        |Author_2|231.97       |
|7        |Author_7|132.66       |
|1        |Author_1|111.86       |
|8        |Author_8|107.16       |
|5        |Author_5|88.83        |
+---------+--------+-------------+

Количество книг в каждом жанре:
+-----------+-----------+
|genre      |books_count|
+-----------+-----------+
|Non-Fiction|9          |
|Science    |3          |
|Fiction    |3          |
|Fantasy    |3          |
|Mystery    |2          |
+-----------+-----------+

Среднюя цена книги по каждому автору:
+---------+---------+---------+
|author_id|name     |avg_price|
+---------+---------+---------+
|5        |Author_5 |88.83    |
|4        |Author_4 |83.7     |
|2        |Author_2 |57.99    |
|9        |Author_9 |46.31    |
|7        |Author_7 |44.22    |
|6        |Author_6 |43.97    |
|1        |Author_1 |37.29    |
|